# SQL Injection - Hands-On Lab

**Part of HackLearn Pro**

Welcome to this interactive lab on SQL Injection! Learn how attackers exploit database vulnerabilities and how to write secure database code.

## Learning Objectives
- Understand what SQL injection is and why it's dangerous
- Explore different types of SQL injection attacks
- Learn to identify vulnerable code patterns
- Practice writing secure database queries
- Implement defense mechanisms

## Prerequisites
- Basic Python knowledge
- Understanding of SQL queries
- Familiarity with databases

---

## Setup

We'll use SQLite for this lab - a lightweight database that's perfect for learning:

In [ ]:
import sqlite3
import hashlib
from typing import Optional, List, Dict, Tuple
import re

print("Setup complete! Ready to explore SQL injection.")

## Part 1: Setting Up a Vulnerable Database

Let's create a simple user database to demonstrate SQL injection:

In [ ]:
def create_vulnerable_database():
    """Create a sample database with user accounts"""
    conn = sqlite3.connect(':memory:')  # In-memory database
    cursor = conn.cursor()
    
    # Create users table
    cursor.execute('''
        CREATE TABLE users (
            id INTEGER PRIMARY KEY,
            username TEXT NOT NULL,
            password TEXT NOT NULL,
            email TEXT,
            role TEXT DEFAULT 'user',
            balance REAL DEFAULT 0.0
        )
    ''')
    
    # Insert sample users
    users = [
        ('admin', 'SecretPass123!', 'admin@hacklearn.com', 'admin', 10000.0),
        ('alice', 'alice1234', 'alice@example.com', 'user', 500.0),
        ('bob', 'bobsecret', 'bob@example.com', 'user', 750.0),
        ('charlie', 'charlie99', 'charlie@example.com', 'user', 250.0),
    ]
    
    cursor.executemany(
        'INSERT INTO users (username, password, email, role, balance) VALUES (?, ?, ?, ?, ?)',
        users
    )
    
    # Create products table
    cursor.execute('''
        CREATE TABLE products (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            stock INTEGER DEFAULT 0
        )
    ''')
    
    products = [
        ('Laptop', 999.99, 10),
        ('Mouse', 29.99, 50),
        ('Keyboard', 79.99, 30),
    ]
    
    cursor.executemany(
        'INSERT INTO products (name, price, stock) VALUES (?, ?, ?)',
        products
    )
    
    conn.commit()
    return conn

# Create the database
db = create_vulnerable_database()
print("✓ Vulnerable database created!")

# Display users
cursor = db.cursor()
cursor.execute('SELECT id, username, email, role, balance FROM users')
users = cursor.fetchall()

print("\nDatabase Contents:")
print("=" * 70)
print(f"{'ID':<5} {'Username':<12} {'Email':<25} {'Role':<10} {'Balance':>10}")
print("=" * 70)
for user in users:
    print(f"{user[0]:<5} {user[1]:<12} {user[2]:<25} {user[3]:<10} ${user[4]:>9.2f}")

## Part 2: Vulnerable Login System

Let's create a vulnerable login function that's susceptible to SQL injection:

In [ ]:
def vulnerable_login(username: str, password: str) -> Optional[Dict]:
    """
    VULNERABLE login function - DO NOT USE IN PRODUCTION!
    Directly concatenates user input into SQL query
    """
    cursor = db.cursor()
    
    # VULNERABLE: Direct string concatenation
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    
    print(f"\nExecuting query: {query}")
    
    try:
        cursor.execute(query)
        result = cursor.fetchone()
        
        if result:
            return {
                'id': result[0],
                'username': result[1],
                'email': result[3],
                'role': result[4],
                'balance': result[5]
            }
        return None
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

# Test with legitimate credentials
print("Test 1: Legitimate Login")
print("=" * 70)
result = vulnerable_login('alice', 'alice1234')
if result:
    print(f"✓ Login successful! Welcome, {result['username']}")
    print(f"  Role: {result['role']}, Balance: ${result['balance']}")
else:
    print("✗ Login failed")

## Part 3: SQL Injection Attacks

### Attack 1: Authentication Bypass

In [ ]:
print("\nTest 2: SQL Injection - Authentication Bypass")
print("=" * 70)

# Injection payload: ' OR '1'='1' --
# This makes the query always return true
malicious_username = "admin' OR '1'='1' --"
malicious_password = "anything"

result = vulnerable_login(malicious_username, malicious_password)

if result:
    print(f"\n🚨 ATTACK SUCCESSFUL!")
    print(f"Logged in as: {result['username']}")
    print(f"Role: {result['role']} (should be admin!)")
    print(f"Balance: ${result['balance']}")
    print("\n⚠️ This demonstrates a critical vulnerability!")
else:
    print("✗ Attack blocked")

### Attack 2: UNION-based Injection

In [ ]:
def vulnerable_search(search_term: str) -> List[Dict]:
    """Vulnerable product search function"""
    cursor = db.cursor()
    
    # VULNERABLE: Direct string interpolation
    query = f"SELECT name, price FROM products WHERE name LIKE '%{search_term}%'"
    
    print(f"\nExecuting query: {query}")
    
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return [{'name': r[0], 'price': r[1]} for r in results]
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return []

print("\nTest 3: UNION-based SQL Injection")
print("=" * 70)

# UNION injection to extract user data
injection_payload = "' UNION SELECT username, password FROM users --"

results = vulnerable_search(injection_payload)

if results:
    print(f"\n🚨 ATTACK SUCCESSFUL! Extracted {len(results)} records:")
    print("\nStolen credentials:")
    for i, result in enumerate(results, 1):
        print(f"{i}. Username: {result['name']}, Password: {result['price']}")
    print("\n⚠️ All user credentials have been exposed!")
else:
    print("No results returned")

### Attack 3: Blind SQL Injection

In [ ]:
def vulnerable_check_username(username: str) -> bool:
    """Check if username exists (vulnerable to blind injection)"""
    cursor = db.cursor()
    
    query = f"SELECT COUNT(*) FROM users WHERE username = '{username}'"
    
    try:
        cursor.execute(query)
        count = cursor.fetchone()[0]
        return count > 0
    except sqlite3.Error as e:
        return False

print("\nTest 4: Blind SQL Injection")
print("=" * 70)

# Blind injection to extract admin password length
print("Attempting to determine admin password length...\n")

for length in range(1, 20):
    # Injection that returns true if password length matches
    payload = f"admin' AND LENGTH(password) = {length} --"
    
    if vulnerable_check_username(payload):
        print(f"🚨 FOUND! Admin password length: {length} characters")
        print("\n⚠️ Attacker can now brute-force character by character!")
        break
else:
    print("Password length not determined in range 1-20")

### Attack 4: Time-based Blind Injection

In [ ]:
import time

def vulnerable_query_with_timing(user_id: str) -> Optional[Dict]:
    """Query user by ID (vulnerable to time-based blind injection)"""
    cursor = db.cursor()
    
    query = f"SELECT username, email FROM users WHERE id = {user_id}"
    
    try:
        start_time = time.time()
        cursor.execute(query)
        result = cursor.fetchone()
        elapsed = time.time() - start_time
        
        return {'result': result, 'time': elapsed}
    except sqlite3.Error as e:
        return None

print("\nTest 5: Time-based Blind SQL Injection")
print("=" * 70)

# Note: SQLite doesn't have SLEEP/WAITFOR, but other databases do
# This demonstrates the concept
print("Concept demonstration:")
print("In MySQL: 1 OR IF(1=1, SLEEP(5), 0) --")
print("In PostgreSQL: 1 OR pg_sleep(5) --")
print("\nThese payloads cause the database to delay response,")
print("allowing attackers to extract data based on timing differences.")

## Part 4: Secure Implementation

Now let's implement secure versions using parameterized queries:

In [ ]:
def secure_login(username: str, password: str) -> Optional[Dict]:
    """
    SECURE login function using parameterized queries
    """
    cursor = db.cursor()
    
    # SECURE: Use parameterized query with placeholders
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    
    print(f"\nExecuting secure query with parameters")
    
    try:
        cursor.execute(query, (username, password))
        result = cursor.fetchone()
        
        if result:
            return {
                'id': result[0],
                'username': result[1],
                'email': result[3],
                'role': result[4],
                'balance': result[5]
            }
        return None
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

print("\nTest 6: Secure Login - Testing with Injection Payload")
print("=" * 70)

# Try the same injection payload
malicious_username = "admin' OR '1'='1' --"
malicious_password = "anything"

result = secure_login(malicious_username, malicious_password)

if result:
    print(f"✗ Login successful (SECURITY FAILURE)")
else:
    print("✓ Login failed - Injection payload treated as literal string")
    print("✓ SECURE: Attack was blocked!")

### Secure Search Implementation

In [ ]:
def secure_search(search_term: str) -> List[Dict]:
    """Secure product search using parameterized queries"""
    cursor = db.cursor()
    
    # SECURE: Parameterized query
    query = "SELECT name, price FROM products WHERE name LIKE ?"
    
    print(f"\nExecuting secure search with parameter: {search_term}")
    
    try:
        # Add wildcards in the parameter, not the query
        cursor.execute(query, (f'%{search_term}%',))
        results = cursor.fetchall()
        return [{'name': r[0], 'price': r[1]} for r in results]
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return []

print("\nTest 7: Secure Search - Testing with UNION Injection")
print("=" * 70)

# Try UNION injection
injection_payload = "' UNION SELECT username, password FROM users --"
results = secure_search(injection_payload)

print(f"\nResults: {len(results)} products found")
if not results:
    print("✓ SECURE: No data returned from injection attempt")
    print("✓ The injection payload was treated as a literal search term")
else:
    for result in results:
        print(f"  - {result['name']}: ${result['price']}")

## Part 5: Input Validation and Sanitization

Additional defense layer: input validation

In [ ]:
class InputValidator:
    """Input validation for database queries"""
    
    # SQL injection patterns to detect
    SQL_PATTERNS = [
        r"(\bOR\b|\bAND\b).*?=.*?['\"]?",  # OR/AND conditions
        r"['\"];?",  # Quotes and semicolons
        r"--",  # SQL comments
        r"/\*.*?\*/",  # Multi-line comments
        r"\bUNION\b.*?\bSELECT\b",  # UNION SELECT
        r"\bDROP\b|\bDELETE\b|\bINSERT\b|\bUPDATE\b",  # Dangerous keywords
        r"\bEXEC\b|\bEXECUTE\b",  # Command execution
        r"xp_cmdshell",  # SQL Server command execution
    ]
    
    @classmethod
    def is_safe(cls, user_input: str) -> Tuple[bool, str]:
        """Check if input is safe from SQL injection"""
        if not user_input:
            return True, "Empty input"
        
        for pattern in cls.SQL_PATTERNS:
            if re.search(pattern, user_input, re.IGNORECASE):
                return False, f"Suspicious pattern detected: {pattern}"
        
        return True, "Input appears safe"
    
    @classmethod
    def sanitize_username(cls, username: str) -> str:
        """Sanitize username input"""
        # Only allow alphanumeric and underscore
        return re.sub(r'[^a-zA-Z0-9_]', '', username)
    
    @classmethod
    def validate_username(cls, username: str) -> Tuple[bool, str]:
        """Validate username format"""
        if not username:
            return False, "Username cannot be empty"
        
        if len(username) < 3:
            return False, "Username must be at least 3 characters"
        
        if len(username) > 32:
            return False, "Username must be 32 characters or less"
        
        if not re.match(r'^[a-zA-Z0-9_]+$', username):
            return False, "Username can only contain letters, numbers, and underscores"
        
        return True, "Valid username"

# Test input validation
print("\nTest 8: Input Validation")
print("=" * 70)

test_inputs = [
    "alice",
    "admin' OR '1'='1' --",
    "bob; DROP TABLE users; --",
    "user123",
    "' UNION SELECT * FROM users --",
]

for test_input in test_inputs:
    is_safe, reason = InputValidator.is_safe(test_input)
    status = "✓ SAFE" if is_safe else "⚠️ BLOCKED"
    
    print(f"\nInput: {test_input}")
    print(f"Status: {status}")
    print(f"Reason: {reason}")

## Part 6: Complete Secure Login System

In [ ]:
def fully_secure_login(username: str, password: str) -> Optional[Dict]:
    """
    Fully secure login with multiple layers of protection
    """
    # Layer 1: Input validation
    is_valid, reason = InputValidator.validate_username(username)
    if not is_valid:
        print(f"✗ Validation failed: {reason}")
        return None
    
    # Layer 2: SQL injection detection
    is_safe_user, _ = InputValidator.is_safe(username)
    is_safe_pass, _ = InputValidator.is_safe(password)
    
    if not (is_safe_user and is_safe_pass):
        print("✗ Security check failed: Suspicious input detected")
        return None
    
    # Layer 3: Parameterized query
    cursor = db.cursor()
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    
    try:
        cursor.execute(query, (username, password))
        result = cursor.fetchone()
        
        if result:
            return {
                'id': result[0],
                'username': result[1],
                'email': result[3],
                'role': result[4],
                'balance': result[5]
            }
        return None
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

print("\nTest 9: Fully Secure Login System")
print("=" * 70)

# Test with legitimate credentials
print("\nTest 9a: Legitimate login")
result = fully_secure_login('alice', 'alice1234')
if result:
    print(f"✓ Login successful: {result['username']}")
else:
    print("✗ Login failed")

# Test with injection payload
print("\nTest 9b: Injection attempt")
result = fully_secure_login("admin' OR '1'='1' --", "anything")
if result:
    print("✗ SECURITY FAILURE: Injection succeeded")
else:
    print("✓ SECURE: Injection blocked")

## Part 7: Challenge Exercises

### Challenge 1: Implement Prepared Statements Class

In [ ]:
class SecureDatabase:
    """
    Database wrapper with built-in SQL injection protection
    
    TODO: Implement secure database operations
    """
    
    def __init__(self, connection):
        self.conn = connection
    
    def execute_safe(self, query: str, params: tuple) -> List:
        """Execute a query safely with parameters"""
        # TODO: Implement safe query execution
        pass
    
    def get_user_by_username(self, username: str) -> Optional[Dict]:
        """Safely retrieve user by username"""
        # TODO: Implement using parameterized query
        pass
    
    def update_user_balance(self, user_id: int, new_balance: float) -> bool:
        """Safely update user balance"""
        # TODO: Implement safe update
        pass

# Test your implementation
# secure_db = SecureDatabase(db)
# user = secure_db.get_user_by_username('alice')

### Challenge 2: Build an Advanced WAF (Web Application Firewall)

In [ ]:
class SQLInjectionWAF:
    """
    Web Application Firewall for SQL injection detection
    
    TODO: Implement advanced detection
    """
    
    def __init__(self):
        self.blocked_attempts = []
        self.whitelist = set()
    
    def detect_sqli(self, user_input: str) -> Tuple[bool, List[str]]:
        """Detect SQL injection attempts"""
        # TODO: Implement comprehensive detection
        # Hints:
        # - Check for SQL keywords
        # - Detect quote patterns
        # - Analyze comment markers
        # - Look for UNION attacks
        # - Detect time-based patterns
        pass
    
    def log_attempt(self, user_input: str, threat_level: str):
        """Log blocked attempts"""
        # TODO: Implement logging
        pass

# Test your WAF
# waf = SQLInjectionWAF()
# is_malicious, reasons = waf.detect_sqli("admin' OR '1'='1' --")

### Challenge 3: Create an Automated Vulnerability Scanner

In [ ]:
class SQLInjectionScanner:
    """
    Automated scanner to detect SQL injection vulnerabilities
    
    TODO: Implement vulnerability scanner
    """
    
    def __init__(self):
        self.payloads = [
            "' OR '1'='1' --",
            "' UNION SELECT NULL --",
            "1' AND '1'='1",
            # Add more payloads
        ]
    
    def scan_login_form(self, login_function) -> Dict:
        """Test login function for vulnerabilities"""
        # TODO: Test with various payloads
        # Return vulnerability report
        pass
    
    def scan_search_function(self, search_function) -> Dict:
        """Test search function for vulnerabilities"""
        # TODO: Test search with injection payloads
        pass

# Test your scanner
# scanner = SQLInjectionScanner()
# report = scanner.scan_login_form(vulnerable_login)
# print(report)

## Summary & Key Takeaways

In this lab, you learned:

1. **SQL Injection Types**:
   - Classic injection (authentication bypass)
   - UNION-based injection (data extraction)
   - Blind injection (information inference)
   - Time-based blind injection

2. **Vulnerabilities**:
   - String concatenation in queries
   - Unvalidated user input
   - Lack of input sanitization

3. **Defense Strategies**:
   - **Always use parameterized queries** (prepared statements)
   - Input validation and sanitization
   - Principle of least privilege
   - Web Application Firewall (WAF)
   - Regular security audits

### Best Practices
1. Never concatenate user input into SQL queries
2. Use ORM frameworks when possible
3. Validate and sanitize all user input
4. Use stored procedures with parameters
5. Implement proper error handling (don't expose DB errors)
6. Apply principle of least privilege to DB accounts
7. Keep databases and frameworks updated
8. Use WAF to detect and block attacks

### Real-World Impact
- Data breaches exposing millions of users
- Financial theft from banking systems
- Complete database compromise
- Regulatory fines and legal consequences

### Further Reading
- [OWASP SQL Injection](https://owasp.org/www-community/attacks/SQL_Injection)
- [SQLMap Tool](http://sqlmap.org/)
- [Bobby Tables](https://bobby-tables.com/) - Guide to preventing SQL injection

---

**HackLearn Pro** - Learn by doing, secure by design.
